In [71]:
import numpy as np
import pandas as pd
import datetime
import os
import seaborn as sns
import matplotlib.pyplot as plt
from patsy import dmatrices, dmatrix
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier

# os.getcwd() 
# os.chdir("/Users/macbook/Downloads/lending-club")
accepted = pd.read_csv('accepted_2007_to_2018Q4.csv')

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# sample = accepted.sample(20)
# print(sample.head(5))

In [49]:
# accepted = accepted.sample(10000)

In [72]:
# convert to Datetime

accepted['issue_d'] = pd.to_datetime(accepted['issue_d'])
accepted['last_pymnt_d'] = pd.to_datetime(accepted['last_pymnt_d'])
accepted['last_credit_pull_d'] = pd.to_datetime(accepted['last_credit_pull_d'])
accepted['next_pymnt_d'] = pd.to_datetime(accepted['next_pymnt_d'])
accepted['debt_settlement_flag_date'] = pd.to_datetime(accepted['debt_settlement_flag_date'])
accepted['settlement_date'] = pd.to_datetime(accepted['settlement_date'])
accepted['hardship_start_date'] = pd.to_datetime(accepted['hardship_start_date'])
accepted['hardship_end_date'] = pd.to_datetime(accepted['hardship_end_date'])
accepted['payment_plan_start_date'] = pd.to_datetime(accepted['payment_plan_start_date'])
accepted['debt_settlement_flag_date'] = pd.to_datetime(accepted['debt_settlement_flag_date'])
accepted['settlement_date'] = pd.to_datetime(accepted['settlement_date'])

In [73]:
accepted['loan_status'].value_counts()

Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [74]:
accepted.shape

(2260701, 151)

In [75]:
# remove all the loans with status == current 

accepted = accepted[accepted.loan_status != 'Current']

In [76]:
accepted.shape

(1382384, 151)

In [77]:
# print(sample.columns[:30])
# print(sample.columns[31:60])
# print(sample.columns[61:90])
# print(sample.columns[91:120])
# print(sample.columns[121:])

In [78]:
# 33 rows out of the whole dataframe has all NaN values, drop

rows = list(accepted[['loan_amnt','annual_inc','zip_code','inq_last_6mths']])

for row in rows:
    accepted = accepted.dropna(axis=0, subset=[row])

In [79]:
# Clean up the dataframe

# Make my outcome variable 0s and 1s
# outcome_var = []
# for i in accepted['grade']:
#     if i == 'A':
#         outcome_var.append(1)
#     else:
#         outcome_var.append(0)
# accepted['outcome_var'] = outcome_var

# Find all the mths_since columns
mths_since_list = [i for i in accepted.columns if 'mths_since' in i]

# Bin the mths_since columns to make them valid inputs
for col in mths_since_list:
    if col != 'sec_app_mths_since_last_major_derog':
        category = list(pd.cut(accepted[col], bins=4, labels=['0', '1', '2', '3']))
        for i, val in enumerate(category):
            if type(val) != str:
                if np.isnan(val):
                    category[i] = '4'
    accepted[col + '_cat'] = category

# Replace nulls in columns with strings (i.e. if no job title, replace null with 'none')
ok_list = ['emp_title','emp_length']
for col in ok_list:
    new_list = []
    col_values = list(accepted[col])
    for i, val in enumerate(col_values):
        if type(val) != str:
            if np.isnan(val):
                new_list.append('none')
            else:
                new_list.append(val)
        else:
            new_list.append(val)
    new_col_name = col + '_2'
    accepted[new_col_name] = new_list
    
emp_length_int = []
for i in accepted['emp_length_2']:
    if i == 'none':
        emp_length_int.append(0)
    else:
        emp_length_int.append(int(re.findall(r'\d+', i)[0]))
accepted['emp_length_int'] = emp_length_int

earliest_cr_yrs = []
for i in accepted['earliest_cr_line']:
    earliest_cr_yrs.append(2019 - int(re.findall(r'\d+', i)[0]))
accepted['earliest_cr_yrs'] = earliest_cr_yrs

own_home = []
for i in accepted['home_ownership']:
    if i == 'OWN':
        own_home.append(1)
    else:
        own_home.append(0)
accepted['own_home'] = own_home

In [80]:
means = list(accepted[['dti','revol_util','tot_cur_bal','all_util','total_bal_il','tot_coll_amt','avg_cur_bal',
                  'bc_open_to_buy','bc_util','total_rev_hi_lim','il_util','max_bal_bc','pct_tl_nvr_dlq',
                  'tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit']])

for mean in means:
    accepted[mean].fillna((accepted[mean].mean()), inplace=True)

In [81]:
modes = list(accepted[['open_acc_6m','open_act_il','open_il_12m','collections_12_mths_ex_med','num_actv_rev_tl',
                 'open_rv_12m','open_rv_24m','open_act_il','num_actv_rev_tl','open_rv_24m','inq_fi','inq_last_12m',
                 'acc_open_past_24mths','num_actv_bc_tl','num_actv_rev_tl','num_bc_tl','num_il_tl','num_op_rev_tl',
                 'num_rev_tl_bal_gt_0','num_sats','percent_bc_gt_75']])

for mode in modes:
    accepted[mode].fillna((accepted[mode].mode()[0]), inplace=True) 

In [82]:
zeros = list(accepted[['open_acc_6m','open_il_12m','collections_12_mths_ex_med','open_rv_12m','total_cu_tl',
                  'chargeoff_within_12_mths','num_accts_ever_120_pd','num_rev_accts','num_tl_120dpd_2m',
                 'num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m','pub_rec_bankruptcies','tax_liens']])

for zero in zeros:
    accepted[zero] = accepted[zero].replace(np.nan, 0)

In [83]:
accepted.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,mths_since_last_delinq_cat,mths_since_last_record_cat,mths_since_last_major_derog_cat,mths_since_rcnt_il_cat,mths_since_recent_bc_cat,mths_since_recent_bc_dlq_cat,mths_since_recent_inq_cat,mths_since_recent_revol_delinq_cat,sec_app_mths_since_last_major_derog_cat,emp_title_2,emp_length_2,emp_length_int,earliest_cr_yrs,own_home
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,2015-12-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,2019-01-01,122.67,NaT,2019-03-01,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaT,NaN,NaT,NaN,NaN,NaN,0,4,0,0,0,1,0,1,1,leadman,10+ years,10,16,0
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,2015-12-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0

In [84]:
accepted['last_pymnt_d'] = accepted['last_pymnt_d'].fillna('NA') 
accepted['next_pymnt_d'] = accepted['next_pymnt_d'].fillna('NA') 
accepted['last_credit_pull_d'] = accepted['last_credit_pull_d'].fillna('NA')

In [85]:
# find all the joint-features

sec_app_list = [i for i in accepted.columns if 'sec_app' in i]
print(sec_app_list)

join_list = [i for i in accepted.columns if 'joint' in i]
print(join_list)

# subset the accounts with joint-features 

sub_joint = accepted[['sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 
                     'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util',
                     'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths',
                     'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 
                     'sec_app_mths_since_last_major_derog_cat','annual_inc_joint', 'dti_joint', 
                     'verification_status_joint', 'revol_bal_joint']]

['sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'sec_app_mths_since_last_major_derog_cat']
['annual_inc_joint', 'dti_joint', 'verification_status_joint', 'revol_bal_joint']


In [86]:
# irrelevant features are dropped

accepted.drop(['id','member_id','desc','title','url','pymnt_plan','debt_settlement_flag_date','settlement_date',
               'payment_plan_start_date', 'hardship_type','hardship_reason','hardship_status','deferral_term',
               'hardship_amount','hardship_start_date','hardship_end_date','hardship_length','hardship_dpd',
               'hardship_loan_status','orig_projected_additional_accrued_interest','hardship_payoff_balance_amount',
               'hardship_last_payment_amount','settlement_status','settlement_amount','settlement_percentage',
               'settlement_term','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op',
               'mo_sin_rcnt_tl'],axis=1,inplace=True)

In [87]:
accepted.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,disbursement_method,debt_settlement_flag,mths_since_last_delinq_cat,mths_since_last_record_cat,mths_since_last_major_derog_cat,mths_since_rcnt_il_cat,mths_since_recent_bc_cat,mths_since_recent_bc_dlq_cat,mths_since_recent_inq_cat,mths_since_recent_revol_delinq_cat,sec_app_mths_since_last_major_derog_cat,emp_title_2,emp_length_2,emp_length_int,earliest_cr_yrs,own_home
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,2015-12-01,Fully Paid,debt_consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,2019-01-01 00:00:00,122.67,NA,2019-03-01 00:00:00,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Cash,N,0,4,0,0,0,1,0,1,1,leadman,10+ years,10,16,0
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,2015-12-01,Fully Paid,small_business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,2016-06-01 00:00:00,926.35,NA,2019-03-01 00:00:00,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Cash,N,0,4,4,0,0,4,0,0,0,Engineer,10+ years,10,20,0
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,2015-12-01,Fully Paid,home_improvement,605xx,IL,10.78,0.0,Aug-2000,695.0,699.0,0.0,NaN,NaN,6.0,0.0,7869.0,56.2,18.0,w,0.0,0.0,22705.924294,22705.92,20000.0,2705.92,0.0,0.0,0.0,2017-06-01 00:00:00,15813.30

In [88]:
accepted.shape

(1382320, 135)

In [89]:
# drop_rows = list(accepted[['loan_amnt','annual_inc','earliest_cr_line','inq_last_6mths','delinq_2yrs',
#                   'earliest_cr_line','inq_last_6mths','open_acc','pub_rec','total_acc']])

# for drop_row in drop_rows:
#     accepted = accepted.dropna(axis=0, subset=[drop_row])

In [90]:
# model_cols = ['outcome_var',
#               'loan_amnt', 'annual_inc', 'own_home',
#               'verification_status', 'issue_d', 'purpose', 'addr_state', 
#               'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec',
#               'revol_bal', 'revol_util_float', 'total_acc', 'collections_12_mths_ex_med',
#               'acc_now_delinq', 'tot_coll_amt', 'total_rev_hi_lim', 'acc_open_past_24mths', 
#               'avg_cur_bal', 'bc_open_to_buy', 'chargeoff_within_12_mths', 'delinq_amnt',
#               'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
#               'mths_since_recent_bc', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
#               'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts',
#               'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',
#               'pct_tl_nvr_dlq', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 
#               'total_bc_limit', 'total_il_high_credit_limit', 'mths_since_last_delinq_cat',
#               'mths_since_last_record_cat', 'mths_since_last_major_derog_cat', 'mths_since_rcnt_il_cat',
#               'mths_since_recent_bc_cat', 'mths_since_recent_bc_dlq_cat', 'mths_since_recent_inq_cat',
#               'mths_since_recent_revol_delinq_cat', 'emp_length_int', 'earliest_cr_yrs']

# model_data = lc_d_2015[model_cols].copy()

In [91]:
accepted['grade'].value_counts()

B    400643
C    394339
A    237477
D    209285
E     97558
F     33315
G      9703
Name: grade, dtype: int64

In [92]:
accepted['term'].value_counts()

 36 months    1043029
 60 months     339291
Name: term, dtype: int64

In [93]:
accepted_36m = accepted[accepted.term != '60 months']

In [94]:
accepted_60m = accepted[accepted.term != '36 months']

In [24]:
# from sklearn import preprocessing

# dummy = pd.get_dummies(accepted['term'])
# accepted = pd.concat([accepted,dummy], axis =1)

# #creating labelEncoder
# le = preprocessing.LabelEncoder()
# # Converting string labels into numbers.

# lists = accepted[['grade','sub_grade','emp_title','emp_length','home_ownership','verification_status','issue_d',
#                   'loan_status','pymnt_plan','desc','purpose','title','zip_code','addr_state','earliest_cr_line',
                 
#                  ]]


# grade_encoded=le.fit_transform(accepted['grade'])
# sub_grade_encoded=le.fit_transform(accepted['sub_grade'])
# emp_title_encoded=le.fit_transform(accepted['emp_title'])
# emp_length_encoded=le.fit_transform(accepted['emp_length'])
# h



In [96]:
accepted.isna().sum()

loan_amnt          0
funded_amnt        0
funded_amnt_inv    0
term               0
int_rate           0
                  ..
emp_title_2        0
emp_length_2       0
emp_length_int     0
earliest_cr_yrs    0
own_home           0
Length: 135, dtype: int64

In [97]:
accepted.shape

(1382320, 135)